# Join dataframes (business + reviews)
## Yelp data
* Joined the business and reviews data in dataframe and stored it in "bus+review_df.p" 
* Method of reading the joined dataframe: major_df = pickle_load("data/bus+review_df.p")
* Column ['review_count'] is updated (we have removed the reviews which are less than 90 days before the last review date)
* Created the column of ['reviews'] which stores all the reviews of a given business_id. Reviews are separated with " "

1\. Import reviews dictionary and yelp_business dataframe

In [2]:
import json
import pandas as pd
import pickle

In [51]:
BUS_PATH = "data/processed/processed_bus_df.p"
bus_df = pd.read_pickle(BUS_PATH)

In [75]:
bus_df.columns

Index(['business_id', 'name', 'neighborhood', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'BikeParking', 'BusinessAcceptsBitcoin', 'BusinessAcceptsCreditCards', 'BusinessParking_garage', 'BusinessParking_street', 'BusinessParking_validated', 'BusinessParking_lot', 'BusinessParking_valet', 'DogsAllowed', 'RestaurantsPriceRange2', 'WheelchairAccessible', 'categories', 'hours', 'type', 'Alcohol', 'Ambience_romantic', 'Ambience_intimate', 'Ambience_classy', 'Ambience_hipster', 'Ambience_touristy', 'Ambience_trendy', 'Ambience_upscale', 'Ambience_casual', 'Caters', 'GoodForKids', 'GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch', 'GoodForMeal_dinner', 'GoodForMeal_breakfast', 'GoodForMeal_brunch', 'HasTV', 'NoiseLevel', 'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsDelivery', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut', 'WiFi', 'AcceptsInsurance',
   

Note: using workaround for issue 24658

In [6]:
file_path = "data/review_cleaned_d.p"

class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size

def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)
    
def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

In [50]:
import pickle
review_d = pickle_load(file_path)

2.Create reviews column in bus_df. Update bus_df['review_count']

In [68]:
import numpy as np
instances_num = len(bus_df['business_id'])  #48493
bus_df['reviews'] = ""
for index, row in bus_df.iterrows():
    if row['business_id'] in review_d:
        text = ""
        review_num = 0
        for key, value in review_d[row['business_id']].items():
            for review in value:
                text = text + review + " "
                review_num += 1
        bus_df.loc[bus_df.business_id == row['business_id'], 'review_count'] = review_num
        bus_df.loc[bus_df.business_id == row['business_id'], 'reviews'] = text
    else:
        print("error: business_id mismatch.")
        
bus_df['review_count']

# Check:
# --6MefnULPED_I942VcFNA      24
# --9e1ONYQuAa-CB_Rrw7Tw    1311
# --DaPTJW3-tB1vP-PfdTEg      32
# --FBCX-N37CMYDfs790Bnw      88

--6MefnULPED_I942VcFNA      22
--9e1ONYQuAa-CB_Rrw7Tw    1280
--DaPTJW3-tB1vP-PfdTEg      29
--FBCX-N37CMYDfs790Bnw      84
--GM_ORV2cYS-h38DSaCLw       5
--I7YYLada0tSLkORTHb5Q      42
--KCl2FvVQpvjzmZSPyviA       8
--Ni3oJ4VOqfOEu7Sj2Vzg       3
--S62v0QgkqQaVUhFnNHrw      15
--SrzpvFLwP_YFwB_Cetow      32
--U98MNlDym2cLn36BBPgQ       2
--cZ6Hhc9F7VkKXxHMVZSQ     251
--g-a85VwrdZJNf0R95GcQ      22
--q7kSBRb0vWC8lSkXFByA       5
--qJNlGWyvPJfBrqwp9c0w      29
--ujyvoQlwVoBgMYtADiLA      54
-01XupAWZEXbdNbxNg5mEg      61
-050d_XIor1NpCuWkbIVaQ     701
-05uZNVbb8DhFweTEOoDVg       0
-06-Fdi30wJx-JA8P7CAng      21
-092wE7j5HZOogMLAh40zA      70
-0CTrPQNiSyClxhdO4HSDQ      16
-0NhdsDJsdarxyDPR523ZQ      20
-0NrB58jqKqJfuUCDupcsw      44
-0Sgh0QlUKVsWosCWJzGqQ      24
-0WegMt6Cy966qlDKhu6jA      53
-0aIra_B6iALlfqAriBSYA      15
-0aOudcaAyac0VJbMX-L1g       8
-0mm8pqBSIOYZQHeo8XnkA      19
-0tgMGl7D9B10YjSN2ujLA      96
-0uEqc2vw1xXtuI_r1xTNg       4
-153AjTW5luZPK4omEujWA       3
-1CI4H3D

Testing

In [80]:
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_rows', None)
# #print(str(bus_df.loc[bus_df.business_id == '--6MefnULPED_I942VcFNA', 'reviews']))
# bus_df['reviews'].head()

In [74]:
bus_df.columns

Index(['business_id', 'name', 'neighborhood', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'BikeParking', 'BusinessAcceptsBitcoin', 'BusinessAcceptsCreditCards', 'BusinessParking_garage', 'BusinessParking_street', 'BusinessParking_validated', 'BusinessParking_lot', 'BusinessParking_valet', 'DogsAllowed', 'RestaurantsPriceRange2', 'WheelchairAccessible', 'categories', 'hours', 'type', 'Alcohol', 'Ambience_romantic', 'Ambience_intimate', 'Ambience_classy', 'Ambience_hipster', 'Ambience_touristy', 'Ambience_trendy', 'Ambience_upscale', 'Ambience_casual', 'Caters', 'GoodForKids', 'GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch', 'GoodForMeal_dinner', 'GoodForMeal_breakfast', 'GoodForMeal_brunch', 'HasTV', 'NoiseLevel', 'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsDelivery', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut', 'WiFi', 'AcceptsInsurance',
   

In [77]:
pickle_dump(bus_df, "data/bus+review_df.p")

writing total_bytes=1513910487...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 1513910487)... done.


In [78]:
major_df = pickle_load("data/bus+review_df.p")

In [79]:
major_df.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,DogsAllowed,RestaurantsPriceRange2,WheelchairAccessible,categories,hours,type,Alcohol,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_touristy,Ambience_trendy,Ambience_upscale,Ambience_casual,Caters,GoodForKids,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_breakfast,GoodForMeal_brunch,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WiFi,AcceptsInsurance,ByAppointmentOnly,HairSpecializesIn,Ambience_divey,DriveThru,attributes,BYOB,BYOBCorkage,Corkage,RestaurantsCounterService,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,CoatCheck,GoodForDancing,HappyHour,Music_dj,Music_background_music,Music_no_music,Music_karaoke,Music_live,Music_video,Music_jukebox,Smoking,AgesAllowed,DietaryRestrictions,Open24Hours,reviews
--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,,"328 Highway 7 E, Chalmers Gate 11, Unit 10",Richmond Hill,ON,L4B 3P7,43.840905,-79.399604,3.0,22,1,True,NaN,True,False,False,False,True,False,NaN,2,True,"Chinese,Restaurants",None,business,beer_and_wine,False,False,False,False,False,False,False,True,NaN,True,False,False,True,True,False,False,False,quiet,False,casual,False,True,True,True,True,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I came here late for dinner with my family, and they just ran out of the roast pork that we want..."
--9e1ONYQuAa-CB_Rrw7Tw,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,The Strip,3355 Las Vegas Blvd S,Las Vegas,NV,89109,36.123183,-115.169190,4.0,1280,1,False,NaN,True,True,False,False,False,True,NaN,4,True,"Steakhouses,Restaurants,Cajun/Creole","Monday 11:30-14:0,Tuesday 11:30-14:0,Wednesday 11:30-14:0,Thursday 11:30-14:0,Friday 11:30-14:0,...",business,full_bar,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,average,False,dressy,False,True,True,True,False,no,NaN,NaN,NaN,False,NaN,NaN,False,yes_corkage,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,"I mainly went for the ceasar salad prepared tableside. I ate in the bar, the bartender was very..."
--DaPTJW3-tB1vP-PfdTEg,--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Corso Italia,1218 Saint Clair Avenue W,Toronto,ON,M6E,43.677807,-79.444674,3.5,29,1,True,NaN,True,False,True,False,False,False,NaN,1,NaN,"Restaurants,Breakfast & Brunch","Monday 7:0-15:0,Tuesday 7:0-15:0,Wednesday 7:0-15:0,Thursday 7:0-15:0,Friday 7:0-16:0,Saturday 7...",business,none,False,False,False,False,False,False,False,True,True,True,False,False,False,False,True,True,True,quiet,False,casual,False,True,False,True,True,free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I came here with some friends after a fun night of dancing, in desperate need of some greasy nou..."
--FBCX-N37CMYDfs790Bnw,--FBCX-N37CMYDfs790Bnw,The Bar At Bermuda & St. Rose,Southeast,11624 Bermuda Rd,Henderson,NV,89052,35.978689,-115.155016,4.0,84,1,True,NaN,True,False,False,False,True,False,NaN,2,True,"American (New),Restaurants,Beer,Wine & Spirits,Food,Bars,Nightlife",None,business,full_bar,False,False,False,False,False,False,False,True,False,False,True,True,False,False,True,True,True,average,False,casual,False,True,False,True,True,free,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,NaN,